# Solution to the Unicef Data Engineer assignment

### TASKS / TODO's and estimation (T-Shirt sizes)
1. Download the Ookla data from Ookla’s S3 bucket - for the most recent 12 months (4 quarters) - Effort: M
    1. Combine the quarterly data into a sensible average (Sum of # devices, Mean of upload / download speeds)
2. Download the most recent Worldpop population data from HDX (Preferably using the HDX API) - Effort: M
    1. Sum the total school age population (5-20yrs old) for each school polygon
3. Merge this data with the existing school polygons geojson - Effort: S
4. Write all this to a new geojson file that [Data Visualisation Specialist] can visualise - Effort: S


#### Notes
- First I want to explore the datasets that I need to use. I want to know what format they are in, how I can access them and what their sizes are. Therefore I immediately install the `geowrangler` library for Python and try to access the Ookla data.
- I also download TIF files from the Humanitarian Data Exchange.
- I also set up a virtual environment locally, so that I can easily manage the desired python version and libraries